<a href="https://colab.research.google.com/github/johnymephisto/Machine-Learning-and-Deep-Learning-Experiments/blob/master/Character_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==1.13.1
!pip install -q textgenrnn

In [0]:
#for getting files from drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import numpy as np
import six
import tensorflow as tf
import time
import os
from textgenrnn import textgenrnn as word_rnn
print(tf.__version__)
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
1.13.1


In [0]:
INPUT_TXT = 'dataset/hpotter.txt'
def transform(txt, pad_to=None):
  # drop any non-ascii characters
    output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
    if pad_to is not None:
        output = output[:pad_to]
        output = np.concatenate([
            np.zeros([pad_to - len(txt)], dtype=np.int32),
            output,
        ])
    return output

def training_generator(seq_len=100, batch_size=1024):
    """A generator yields (source, target) arrays for training."""
    with tf.gfile.GFile(INPUT_TXT, 'r') as f:
        txt = f.read()

    tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
    source = transform(txt)
#     source = txt
    while True:
        offsets = np.random.randint(0, len(source) - seq_len, batch_size)

        # Our model uses sparse crossentropy loss, but Keras requires labels
        # to have the same rank as the input logits.  We add an empty final
        # dimension to account for this.
        yield (
            np.stack([source[idx:idx + seq_len] for idx in offsets]),
            np.expand_dims(
                np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
                -1),
        )

six.next(training_generator(seq_len=10, batch_size=1))

(array([[ 44,  32, 119, 105, 116, 104,  32,  97,  32, 112]], dtype=int32),
 array([[[ 32],
         [119],
         [105],
         [116],
         [104],
         [ 32],
         [ 97],
         [ 32],
         [112],
         [117]]], dtype=int32))

In [0]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
    
    """Language model: predict the next word given the current word."""
    
    source = tf.keras.Input(name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

    embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
    
    lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
    
    lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
    
    predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
    
    model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
    
    model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
    return model

In [0]:
tf.keras.backend.clear_session()

training_model = lstm_model(seq_len=300, batch_size=128, stateful=False)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.fit_generator(
    training_generator(seq_len=300, batch_size=1024),
    steps_per_epoch=100,
    epochs=100,
)
tpu_model.save_weights('Saved_Model/potter_1.h5', overwrite=True)

In [0]:
# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=1, stateful=True)
prediction_model.load_weights('gdrive/My Drive/Saved_Model/potter.h5')

In [0]:
BATCH_SIZE = 1
PREDICT_LEN = 200
seed_txt = 'Harry took out his wand and'
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
    prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
    last_word = predictions[-1]
    next_probits = prediction_model.predict(last_word)[:, 0, :]

    # sample from our output distribution
    next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
    ]
    predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
    print('PREDICTION %d\n\n' % i)
    p = [predictions[j][i] for j in range(PREDICT_LEN)]
    generated = ''.join([chr(c) for c in p])
    print(generated)
    print()
    assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


d said, stripping through the impression and umbre to step from the ramp causing, carved around the same, and think she catch his hand she held across the room apart from the twinse faces in the effor



In [0]:
#word model
model_name = 'potter_model_3'
word_gen = word_rnn(weights_path='Saved_Model/{}_weights.hdf5'.format(model_name),
                       vocab_path='Saved_Model/{}_vocab.json'.format(model_name),
                       config_path='Saved_Model/{}_config.json'.format(model_name))

word_gen.generate(max_gen_length=200, temperature=[0.2], prefix=None)

In [0]:
INPUT_TXT = 'gdrive/My Drive/Saved_Model/elon.txt'
def transform(txt, pad_to=None):
  # drop any non-ascii characters
    output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
    if pad_to is not None:
        output = output[:pad_to]
        output = np.concatenate([
            np.zeros([pad_to - len(txt)], dtype=np.int32),
            output,
        ])
    return output

def training_generator(seq_len=100, batch_size=1024):
    """A generator yields (source, target) arrays for training."""
    with tf.gfile.GFile(INPUT_TXT, 'r') as f:
        txt = f.read()

    tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
    source = transform(txt)
    while True:
        offsets = np.random.randint(0, len(source) - seq_len, batch_size)

        # Our model uses sparse crossentropy loss, but Keras requires labels
        # to have the same rank as the input logits.  We add an empty final
        # dimension to account for this.
        yield (
            np.stack([source[idx:idx + seq_len] for idx in offsets]),
            np.expand_dims(
                np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
                -1),
        )

six.next(training_generator(seq_len=10, batch_size=1))

(array([[ 64, 107, 114, 112,  53,  32,  64, 118, 105, 110]], dtype=int32),
 array([[[107],
         [114],
         [112],
         [ 53],
         [ 32],
         [ 64],
         [118],
         [105],
         [110],
         [ 99]]], dtype=int32))

In [0]:
tf.keras.backend.clear_session()

training_model = lstm_model(seq_len=100, batch_size=128, stateful=False)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.fit_generator(
    training_generator(seq_len=100, batch_size=1024),
    steps_per_epoch=100,
    epochs=100,
)
tpu_model.save_weights('gdrive/My Drive/Saved_Model/elon_1.h5', overwrite=True)

In [0]:
# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=1, stateful=True)
prediction_model.load_weights('gdrive/My Drive/Saved_Model/elon.h5')

In [0]:
BATCH_SIZE = 1
PREDICT_LEN = 200
seed_txt = 'My lil monster tesla Model 3 is the'
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
    prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
    last_word = predictions[-1]
    next_probits = prediction_model.predict(last_word)[:, 0, :]

    # sample from our output distribution
    next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
    ]
    predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
    print('PREDICTION %d\n\n' % i)
    p = [predictions[j][i] for j in range(PREDICT_LEN)]
    generated = ''.join([chr(c) for c in p])
    print(generated)
    print()
    assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


e best. Heless reliable. @Model3Owners There is a successful launch from Padie  RT @thetrKBD: .@elonmusk holding access to the video unless sent to us by owners next week. We are just being higher. I 



In [0]:
#word model
model_name = 'elon_model_1'
word_gen = word_rnn(weights_path='gdrive/My Drive/Saved_Model/{}_weights.hdf5'.format(model_name),
                       vocab_path='gdrive/My Drive/Saved_Model/{}_vocab.json'.format(model_name),
                       config_path='gdrive/My Drive/Saved_Model/{}_config.json'.format(model_name))

word_gen.generate(max_gen_length=50, temperature=[0.2], prefix=None)
